In [ ]:
from faker import Faker
fake = Faker('pt_BR')
import datetime, json, requests, random
from dateutil.relativedelta import relativedelta
import azure.cosmos.cosmos_client as cosmos_client
import azure.cosmos.exceptions as exceptions
from azure.cosmos.partition_key import PartitionKey


HOST = 'https://padariacasedb.documents.azure.com:443/'
MASTER_KEY = 'KPxz6QHRsC0rnSC0A2Wi4C2PNVapVy0JU9GpCyUC4LRrD9Xqrmh18BCShyvXTHOrUxyNIFEf848tHGmv221eGA=='
DATABASE_ID = 'padariadb'
CONTAINER_ID = 'products'

In [ ]:
client = cosmos_client.CosmosClient(HOST, {'masterKey': MASTER_KEY} )
db = client.create_database_if_not_exists(id=DATABASE_ID)
container = db.create_container_if_not_exists(
    id=CONTAINER_ID, 
    partition_key=PartitionKey(path="/productId"),
    offer_throughput=400
)
container_provider = db.create_container_if_not_exists(
    id='providers', 
    partition_key=PartitionKey(path="/providerId"),
    offer_throughput=400
)

In [ ]:
items_providers = list(container_provider.query_items(
    query="SELECT top 100 * FROM providers p where RAND() <= @id",
    parameters=[
        { "name":"@id", "value": .009 }
    ],
    enable_cross_partition_query=True
))

In [ ]:
for item in items_providers:
    print(item.get("providerId"))
    print (item.get('summary'))
    item.get('summary')['totalProducts'] = item.get('summary')['totalProducts'] + 1
    #container_provider.upsert_item(body=item)

In [ ]:
items_providers = list(container_provider.query_items(
    query="SELECT top 1000 * FROM providers p where RAND() <= @id",
    parameters=[
        { "name":"@id", "value": .3 }
    ],
    enable_cross_partition_query=True
))
x = 1
category = ["frios", "confeitaria", "lanches","salgados","mercado","pães","tortas","doces","gelados","laticínios","bebidas frias","bebidas quentes","refeição"]
for item in items_providers:
    n = random.randint(0,len(category)-1)
    date = fake.date_between(start_date='-30y', end_date='today')
    expirationdate = fake.date_between(start_date='today', end_date='+10y')
    Product = {
        "id": fake.hexify(text='^^^^-^^^^-^^^^-^^^^'),
        "productId": x,
        "description": "produto " + str(x),
        "providerId": item.get("providerId"),
        "originBranchId": fake.random_number(3),
        "createdDate": date.strftime('%Y-%m-%d'),
        "category": category[n],
        "enabled": True,
        "unit": "UN",
        "summary":{
            "lastQuantity": 0,
            "lastPrice": 0,
            "laststockEntry": "null",
            "lastOrderate":  "null",
            "salesAmount": 0,
            "totalSales": 0.00
        }
        }
    container.create_item(body=Product)
    item.get('summary')['totalProducts'] = item.get('summary')['totalProducts'] + 1
    container_provider.upsert_item(body=item)
    x=x+1
    print(Product)

In [ ]:
#deletar todos fornecedores
item_list = list(container.read_all_items(max_item_count=20))

print('Found {0} items'.format(item_list.__len__()))

for doc in item_list:
    print(doc.get('productId'))
    container.delete_item(item=doc.get('id'), partition_key=doc.get('productId'))